In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime
from itertools import product
from model import NeuralNet
from simulated_annealing import SA, plot_histories
from scipy.special import softmax
from sklearn.datasets import load_breast_cancer
from sklearn.neural_network import MLPClassifier
from tqdm.notebook import tqdm

np.set_printoptions(precision=4, suppress=True)
np.random.seed(0)

In [2]:
dataset = pd.read_excel('./datasets/credit-card-clients.xls', skiprows=1).iloc[:, 1:]
dataset = dataset[dataset['EDUCATION'].isin([1, 2, 3, 4])]
dataset = dataset[dataset['MARRIAGE'].isin([1, 2, 3])]
dataset = pd.get_dummies(dataset, columns=['SEX', 'EDUCATION', 'MARRIAGE'], drop_first=True)

data_norm = (dataset - dataset.min()) / (dataset.max() - dataset.min())
y = data_norm.pop('default payment next month')

print(f'samples: {data_norm.shape[0]}')
data_norm.head()

samples: 29601


,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX_2,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_2,MARRIAGE_3
0,0.010101,0.051724,0.4,0.4,0.1,0.1,0.0,0.0,0.149982,0.069164,...,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0,0.0
1,0.111111,0.086207,0.1,0.4,0.2,0.2,0.2,0.4,0.148892,0.067858,...,0.001116,0.001610,0.000000,0.003783,1.0,1.0,0.0,0.0,1.0,0.0
2,0.080808,0.224138,0.2,0.2,0.2,0.2,0.2,0.2,0.172392,0.079532,...,0.001116,0.001610,0.002345,0.009458,1.0,1.0,0.0,0.0,1.0,0.0
3,0.040404,0.275862,0.2,0.2,0.2,0.2,0.2,0.2,0.188100,0.111995,...,0.001339,0.001771,0.002506,0.001892,1.0,1.0,0.0,0.0,0.0,0.0
4,0.040404,0.620690,0.1,0.2,0.1,0.2,0.2,0.2,0.154144,0.071601,...,0.011160,0.014493,0.001615,0.001284,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
data_norm, y = data_norm.values, y.values.astype('int')

model = NeuralNet(data=data_norm, y=y, hidden_units=2)    # Criando o modelo de Rede Neural
num_iter = 5                                              # Número de execuções para cada algoritmo

print(f'Quantidade de parâmetros da rede: {model.size(model.sample_weights())}')

Quantidade de parâmetros da rede: 60


In [11]:
# Parâmetros da Rede Neural a partir do SGD
backprop_loss, backprop_exec_time = np.zeros(num_iter), []

for i in range(num_iter):
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(128,), solver='adam', alpha=0.0, max_iter=1000, verbose=False)
    start = datetime.now()
    mlp_classifier.fit(data_norm, y)
    backprop_exec_time.append(datetime.now() - start)
    
    p = mlp_classifier.predict_proba(data_norm)
    backprop_loss[i] = model.cross_entropy(y, p)

In [12]:
lbfgs_loss

array([488.7982, 489.0422, 491.4336, 486.0003, 487.9787])

In [ ]:
# Parâmetros da Rede Neural a partir do Simulated Annealing
N_options = [int(1e4), int(1e5)]
T0_options = [10., 1., 1e-1]
K_options =  [6, 8, 12, 16]

In [ ]:
results = []
start_datetime = str(datetime.now()).split('.')[0].replace(' ', '_').replace(':', '').replace('-', '')

for N, T0, K in product(N_options, T0_options, K_options):
    histories = {}
    eps = 0.01
    for i in tqdm(range(num_iter), desc=f'T0 = {T0}, K = {K}, N = {N}'):
        J_min, x_min, history, exec_time = SA(model, N=N, K=K, T0=T0, eps=eps)
        results.append([N, K, T0, eps, J_min, x_min, exec_time])
        if i == 0: histories['T'] = history[1, :]
        histories[f'iter {i+1}'] = history[0, :]
    hists = pd.DataFrame(histories)
    try: os.mkdir(f".\\results\\Breast-Cancer_{start_datetime}")
    except: pass
    hists.to_pickle(f".\\results\\Breast-Cancer_{start_datetime}\\histories_T0-{T0}_K-{K}_N-{N}.pkl")
        
df = pd.DataFrame(results, columns=['N', 'K', 'T0', 'eps', 'J mínimo', 'x mínimo', 'Tempo de Execução'])
df.to_pickle(f".\\results\\Breast-Cancer_{start_datetime}\\results.pkl")

In [ ]:
# start_datetime = '20220507_143854'

df = pd.read_pickle(f'.\\results\\Breast-Cancer_{start_datetime}\\results.pkl')

# L-BFGS
print(f'Backprop, J_mínimo: {np.round(lbfgs_loss.mean(), 4)} +/- {np.round(lbfgs_loss.std(), 4)}, Tempo de execução médio: {np.mean(lbfgs_exec_time)}')
print()

# Simulated Annealing
for N, T0, K in product(N_options, T0_options, K_options):
    # Plot optimization history
    histories = pd.read_pickle(f'.\\results\\Breast-Cancer_{start_datetime}\\histories_T0-{T0}_K-{K}_N-{N}.pkl')
    plot_histories(histories, T0, K, N, savefig=True, dt=start_datetime)
    plt.close()

    # Plot results
    filter_ = (df['T0'] == T0) & (df['K'] == K) & (df['N'] == N)
    print(f'T0: {T0}, K: {K}, N: {N}', end=', ')
    J_mean, J_std = df[filter_]['J mínimo'].mean(), df[filter_]['J mínimo'].std()
    t_mean = df[filter_]['Tempo de Execução'].mean().to_pytimedelta()

    print(f"J_mínimo: {np.round(J_mean, 4)} +/- {np.round(J_std, 4)}", end=', ')
    print(f"Tempo de execução médio: {t_mean}")

In [ ]:
mean_results = df.groupby(['N', 'K', 'T0', 'eps'], as_index=False)[['J mínimo', 'Tempo de Execução']].mean(numeric_only=False).sort_values(by='J mínimo', ignore_index=True)
mean_results

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

data, labels = [], []
columns = ['SA', 'K', 'T0', 'N']
params = []

for idx, row in mean_results.iloc[:5, :].iterrows():
    N, K, T0 = row.N, row.K, row.T0
    data.append(df[(df['N'] == N) & (df['K'] == K) & (df['T0'] == T0)]['J mínimo'].values)
    labels.append(f'SA {idx+1}')
    params.append([idx+1, K, T0, N])

ax.boxplot([lbfgs_loss.tolist()] + data, labels=['Backprop'] + labels)
ax.grid()
ax.set_ylabel('J mínimo')
ax.set_xlabel('Algoritmo')
plt.show()

pd.DataFrame(params, columns=columns).set_index('SA')

In [ ]:
bests = []
for N, T0, K in product(N_options, T0_options, K_options):
    idx = df[(df['N'] == N) & (df['T0'] == T0) & (df['K'] == K)]['J mínimo'].idxmin()
    bests.append(df.iloc[idx, :].tolist())
bests = pd.DataFrame(bests, columns=df.columns).sort_values(by='J mínimo', ignore_index=True)
bests